# Init

In [1]:
cd ..

/Users/wliao0504/code/team_02


In [2]:
pwd

'/Users/wliao0504/code/team_02'

## Load

In [3]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
from google.cloud import bigquery

In [4]:
# Configuration
PROJECT_ID = "sepsis-nlp"

# Initialize BigQuery client with Application Default Credentials
client = bigquery.Client(project=PROJECT_ID)

print(f"Successfully connected to BigQuery project: {PROJECT_ID}")

def run_query(sql):
    """Run a SQL query and return results as DataFrame"""
    query_job = client.query(sql)
    return query_job.to_dataframe()

def list_datasets():
    """List all datasets in the project"""
    datasets = list(client.list_datasets())
    if datasets:
        print("Datasets in project:")
        for dataset in datasets:
            print(f"  - {dataset.dataset_id}")
    else:
        print("No datasets found in project")
    return datasets

# List available datasets
list_datasets()

Successfully connected to BigQuery project: sepsis-nlp
Datasets in project:
  - team_2


## Load mappings

In [130]:
procedure_mapping = pd.read_csv("data/rediscover_concept_counts - procedure.csv")

In [6]:
device_mapping = pd.read_csv("data/rediscover_concept_counts - device_exposure.csv")

## Load cohort

In [132]:
cohort = pd.read_csv("data/cohort.csv")

# Procedure

In [133]:
procedure_mapping

,concept_id,concept_name,n_people,pct_people,records_per_peron,mapping,note
0,4155151,Oxygen administration by nasal cannula,"9,066",7.55 %,11.9,NaN,NaN
1,2213572,Hemodialysis procedure with single evaluation ...,"4,202",3.50 %,34.7,NaN,this is just a charge
2,4230167,Artificial ventilation,"4,112",3.43 %,5.6,NaN,NaN
3,2213576,Dialysis procedure other than hemodialysis (eg...,"3,707",3.09 %,19.2,NaN,this is just a charge
4,2745444,"Insertion of Endotracheal Airway into Trachea,...","3,602",3.00 %,1.2,NaN,NaN
5,4201025,High concentration oxygen therapy,"2,969",2.47 %,12.2,NaN,NaN
6,4162736,Oxygen administration by mask,"2,747",2.29 %,9.4,NaN,NaN
7,44790731,Oxygen administration by non rebreather mask,"2,290",1.91 %,4.9,NaN,NaN
8,37018292,Continuous renal replacement therapy,"1,852",1.54 %,18.9,dialysis,NaN
9,40486624,Non-invasive positive pressure ventilation,"1,701",1.42 %,9.6,NaN,NaN


In [137]:
procedures_ids = {
    "ecmo": procedure_mapping[procedure_mapping['mapping'] == "ECMO"]['concept_id'].tolist(),
    "dialysis": procedure_mapping[procedure_mapping['mapping'] == "dialysis"]['concept_id'].tolist(),
}

In [138]:
procedures_ids["ecmo"]

[4052536, 1531630, 1531631, 1531632, 1524103, 1524104]

In [141]:
query = f"""
SELECT
  c.person_id, c.visit_occurrence_id, c.race_name, c.gender_name,
  MAX(CASE 
    WHEN pr.procedure_concept_id IN ({','.join(map(str, procedures_ids["ecmo"]))}) 
    THEN 1 ELSE 0 END) AS ecmo,
  MAX(CASE WHEN pr.procedure_concept_id IN ({','.join(map(str, procedures_ids["dialysis"]))}) 
    THEN 1 ELSE 0 END) AS dialysis,
FROM `sccm-discovery.rediscover_datathon_2025.cohort_team_2` c
LEFT JOIN `sccm-discovery.rediscover_datathon_2025.procedure_occurrence` pr
  ON c.person_id = pr.person_id
  AND c.visit_occurrence_id = pr.visit_occurrence_id
  AND pr.procedure_concept_id IN ({','.join(map(str, procedures_ids["ecmo"] + procedures_ids["dialysis"]))})
GROUP BY c.person_id, c.visit_occurrence_id, c.race_name, c.gender_name
ORDER BY c.person_id, c.visit_occurrence_id
"""
cohort_procedures = run_query(query)
cohort_procedures

,person_id,visit_occurrence_id,race_name,gender_name,ecmo,dialysis
0,1,53398,White,FEMALE,0,0
1,2,63189,Not Reported,FEMALE,0,0
2,3,65156,White,FEMALE,0,0
3,4,58986,White,FEMALE,0,0
4,5,53241,White,FEMALE,0,0
...,...,...,...,...,...,...
99051,104565,166212,White,MALE,0,0
99052,104566,178432,White,MALE,0,0
99053,104567,169186,White,MALE,0,0
99054,104568,156553,White,MALE,0,0


In [140]:
query = f"""
SELECT 
    race_name,
    AVG(dialysis) as avg_dialysis_ever
FROM cohort_procedures
GROUP BY race_name
"""
dialysis_by_race = duckdb.sql(query).df()
dialysis_by_race

,race_name,avg_dialysis_ever
0,Native Hawaiian or Other Pacific Islander,0.014337
1,American Indian or Alaska Native,0.020408
2,White,0.016446
3,Not Reported,0.061722
4,Asian,0.024101
5,Black,0.042069
6,Mixed,0.000000


## Archive

In [50]:
mask = procedure_mapping['mapping'] == 'dialysis'
dialysis_concept_ids = procedure_mapping[mask]['concept_id'].tolist()
dialysis_concept_ids

[2213572, 2213576, 37018292, 4120120, 2213573, 4146536, 4324124]

In [51]:
# Calculate average number of dialysis procedures per patient by race
query = f"""
WITH dialysis_counts AS (
  SELECT 
    po.person_id,
    p.race_concept_id,
    COUNT(*) as dialysis_count
  FROM `sccm-discovery.rediscover_datathon_2025.procedure_occurrence` po
  JOIN `sccm-discovery.rediscover_datathon_2025.person` p
    ON po.person_id = p.person_id
  WHERE po.procedure_concept_id IN ({','.join(map(str, dialysis_concept_ids))})
  GROUP BY po.person_id, p.race_concept_id
)
SELECT 
  c.concept_name as race,
  dc.race_concept_id,
  COUNT(dc.person_id) as total_patients,
  SUM(dc.dialysis_count) as total_procedures,
  ROUND(AVG(dc.dialysis_count), 2) as avg_dialysis_per_patient
FROM dialysis_counts dc
LEFT JOIN `sccm-discovery.rediscover_datathon_2025.concept` c
  ON dc.race_concept_id = c.concept_id
GROUP BY dc.race_concept_id, c.concept_name
ORDER BY avg_dialysis_per_patient DESC
"""

dialysis_by_race = run_query(query)
print("Average Dialysis Procedures per Patient by Race:")
print("=" * 60)
dialysis_by_race


Average Dialysis Procedures per Patient by Race:


,race,race_concept_id,total_patients,total_procedures,avg_dialysis_per_patient
0,White,8527,3332,405816,121.79
1,American Indian or Alaska Native,8657,56,5658,101.04
2,No matching concept,0,805,69490,86.32
3,Black or African American,8516,3338,256522,76.85
4,Asian,8515,160,11052,69.08
5,Unknown racial group,3213232,44,2474,56.23
6,Other,45878142,17,768,45.18
7,Native Hawaiian or Other Pacific Islander,8557,8,322,40.25
8,Race not stated,4190758,5,154,30.80
9,Other Pacific Islander,38003613,3,68,22.67


In [52]:
cohort_person_ids = cohort['person_id'].tolist()

In [53]:
query = f"""
SELECT DISTINCT person_id, visit_occurrence_id
FROM `sccm-discovery.rediscover_datathon_2025.procedure_occurrence` po
-- JOIN `sccm-discovery.rediscover_datathon_2025.concept` c
  -- ON po.procedure_concept_id = c.concept_id
WHERE procedure_concept_id IN ({','.join(map(str, dialysis_concept_ids))})
-- LIMIT 10;
"""
dialysis_cohort = run_query(query)
dialysis_cohort

,person_id,visit_occurrence_id
0,213,62694
1,461,54419
2,489,62696
3,1366,56562
4,1366,51083
...,...,...
21494,117460,<NA>
21495,118709,<NA>
21496,118746,<NA>
21497,118807,<NA>


In [54]:
len(cohort)

99056

In [55]:
cohort.columns

Index(['Unnamed: 0', 'person_id', 'gender_concept_id', 'year_of_birth',
       'month_of_birth', 'day_of_birth', 'birth_datetime', 'race_concept_id',
       'ethnicity_concept_id', 'location_id', 'provider_id', 'care_site_id',
       'person_source_value', 'gender_source_value',
       'gender_source_concept_id', 'race_source_value',
       'race_source_concept_id', 'ethnicity_source_value',
       'ethnicity_source_concept_id', 'race_name', 'gender_name', 'num_visits',
       'visit_occurrence_id', 'visit_concept_id', 'visit_start_date',
       'visit_start_datetime', 'visit_end_date', 'visit_end_datetime',
       'visit_type_concept_id', 'visit_source_value',
       'visit_source_concept_id', 'admitting_source_concept_id',
       'admitting_source_value', 'discharge_to_concept_id',
       'discharge_to_source_value', 'preceding_visit_occurrence_id',
       'first_visit', 'died_in_hospital', 'died_in_30_days'],
      dtype='object')

In [56]:
query = f"""
SELECT 
    *,
    CASE WHEN dc.person_id is NOT NULL THEN 1 ELSE 0 END as has_dialysis
FROM cohort c
LEFT JOIN dialysis_cohort dc
    ON c.person_id = dc.person_id
    AND c.visit_occurrence_id = dc.visit_occurrence_id
"""
dialysis_cohort_joined = duckdb.sql(query).df()
dialysis_cohort_joined

,Unnamed: 0,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,...,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id,first_visit,died_in_hospital,died_in_30_days,person_id_1,visit_occurrence_id_1,has_dialysis
0,0,70926,8507,1925,1,1,1925-01-01,8516,38003564,NaN,...,NaN,NaN,NaN,NaN,False,False,True,<NA>,<NA>,0
1,1,77592,8507,1925,1,1,1925-01-01,8527,38003564,NaN,...,NaN,NaN,NaN,NaN,True,False,False,<NA>,<NA>,0
2,2,71018,8507,1925,1,1,1925-01-01,0,38003564,NaN,...,NaN,NaN,NaN,NaN,False,False,False,<NA>,<NA>,0
3,3,57129,8507,1925,1,1,1925-01-01,8527,38003564,NaN,...,NaN,NaN,NaN,NaN,True,False,False,<NA>,<NA>,0
4,4,55650,8507,1925,1,1,1925-01-01,8527,38003564,NaN,...,NaN,NaN,NaN,NaN,True,False,False,<NA>,<NA>,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99051,98801,80666,8532,2003,4,1,2003-04-01,8527,38003564,NaN,...,NaN,NaN,NaN,NaN,True,True,False,80666,178411,1
99052,98841,83797,8532,2003,8,1,2003-08-01,8527,38003564,NaN,...,NaN,NaN,NaN,NaN,False,True,False,83797,179506,1
99053,98956,81003,8532,2004,11,1,2004-11-01,8527,38003564,NaN,...,NaN,NaN,NaN,NaN,True,False,False,81003,169764,1
99054,98959,61001,8532,2004,11,1,2004-11-01,0,38003563,NaN,...,NaN,NaN,NaN,NaN,True,False,False,61001,135567,1


## Upload

In [142]:
query = f"""
SELECT person_id, visit_occurrence_id, ecmo, dialysis
FROM cohort_procedures
"""
cohort_procedures_final = duckdb.sql(query).df()

# Upload the dataframe to BigQuery
job = client.load_table_from_dataframe(cohort_procedures_final, "sepsis-nlp.team_2.cohort_procedures")

# Wait for the job to complete
job.result()

LoadJob<project=sepsis-nlp, location=US, id=84d51541-9f2e-40b8-a9ad-f5da52498477>

## By race

In [57]:
query = f"""
SELECT 
    race_name,
    AVG(has_dialysis) as avg_dialysis_ever
FROM dialysis_cohort_joined
GROUP BY race_name
"""
dialysis_by_race = duckdb.sql(query).df()
dialysis_by_race

,race_name,avg_dialysis_ever
0,Native Hawaiian or Other Pacific Islander,0.035842
1,Asian,0.038489
2,Mixed,0.000000
3,White,0.038903
4,Not Reported,0.083679
5,Black,0.096019
6,American Indian or Alaska Native,0.070644


In [58]:
query = f"""
SELECT 
    gender_name,
    AVG(has_dialysis) as avg_dialysis_ever
FROM dialysis_cohort_joined
GROUP BY gender_name
"""
dialysis_by_gender = duckdb.sql(query).df()
dialysis_by_gender

,gender_name,avg_dialysis_ever
0,FEMALE,0.050013
1,MALE,0.062611


# Device

In [60]:
EXCLUDED_DEVICE_CONCEPT_IDS = [4164918]

In [61]:
device_concept_ids = device_mapping['concept_id'].tolist()
device_concept_ids

[45760842,
 4224038,
 45768197,
 4139525,
 4222966,
 4145528,
 4138487,
 4138916,
 40493026,
 45768222,
 45768198,
 4145694,
 45767337,
 4322904,
 45760219,
 45759146,
 36715213,
 45759930,
 4138748,
 4137849,
 36715212,
 4164918,
 45768199,
 45761494,
 605913,
 4138614]

In [ ]:
device_mapper = {
    1: "nasal_canula_mask",
    2: "hiflo_oximyzer",
    3: "cpap_bipap",
    4: "mechanical_ventilation",
}

In [86]:
device_ids_by_type = {
    1: device_mapping[device_mapping['Mapping'] == "1"]['concept_id'].tolist(),
    2: device_mapping[device_mapping['Mapping'] == "2"]['concept_id'].tolist(),
    3: device_mapping[device_mapping['Mapping'] == "3"]['concept_id'].tolist(),
    4: device_mapping[device_mapping['Mapping'] == "4"]['concept_id'].tolist(),
}

In [ ]:
query = f"""
SELECT
  de.person_id, de.visit_occurrence_id, race_name, gender_name,
  MAX(CASE 
    WHEN de.device_concept_id IN ({','.join(map(str, device_ids_by_type[1]))}) 
    -- AND de.person_id IS NOT NULL
    THEN 1 ELSE 0 
    
    END) AS nasal_canula_mask,
  MAX(CASE WHEN de.device_concept_id IN ({','.join(map(str, device_ids_by_type[2]))}) 
    -- AND de.person_id IS NOT NULL
    THEN 1 ELSE 0 END) AS hiflo_oximyzer,
  MAX(CASE WHEN de.device_concept_id IN ({','.join(map(str, device_ids_by_type[3]))}) 
    -- AND de.person_id IS NOT NULL
    THEN 1 ELSE 0 END) AS cpap_bipap,
  MAX(CASE WHEN de.device_concept_id IN ({','.join(map(str, device_ids_by_type[4]))}) 
    -- AND de.person_id IS NOT NULL
    THEN 1 ELSE 0 END) AS mechanical_ventialtion
FROM `sccm-discovery.rediscover_datathon_2025.device_exposure` de
RIGHT JOIN `sccm-discovery.rediscover_datathon_2025.cohort_team_2` c
  ON c.person_id = de.person_id
  AND c.visit_occurrence_id = de.visit_occurrence_id
WHERE de.device_concept_id IN ({','.join(map(str, device_concept_ids))})
  AND de.device_concept_id NOT IN ({','.join(map(str, EXCLUDED_DEVICE_CONCEPT_IDS))})
GROUP BY de.person_id, de.visit_occurrence_id, race_name, gender_name
"""
devices = run_query(query)
devices

,person_id,visit_occurrence_id,race_name,gender_name,nasal_canula_mask,hiflo_oximyzer,cpap_bipap,mechanical_ventialtion
0,33454,534,White,MALE,1,0,0,0
1,33902,828,White,MALE,1,1,0,1
2,34129,1607,Black,MALE,1,0,0,0
3,16346,1760,White,FEMALE,1,0,0,0
4,33643,1961,White,MALE,1,0,0,0
...,...,...,...,...,...,...,...,...
59391,68502,124555,White,MALE,1,0,0,0
59392,75179,142086,White,FEMALE,1,0,0,0
59393,103285,149484,White,MALE,1,0,0,0
59394,82638,152238,Black,FEMALE,1,0,0,0


In [112]:
# Fixed query - include all patients, fill 0s for those without devices
query_fixed = f"""
SELECT
  c.person_id, c.visit_occurrence_id, c.race_name, c.gender_name,
  MAX(CASE 
    WHEN de.device_concept_id IN ({','.join(map(str, device_ids_by_type[1]))}) 
    THEN 1 ELSE 0 END) AS nasal_canula_mask,
  MAX(CASE WHEN de.device_concept_id IN ({','.join(map(str, device_ids_by_type[2]))}) 
    THEN 1 ELSE 0 END) AS hiflo_oximyzer,
  MAX(CASE WHEN de.device_concept_id IN ({','.join(map(str, device_ids_by_type[3]))}) 
    THEN 1 ELSE 0 END) AS cpap_bipap,
  MAX(CASE WHEN de.device_concept_id IN ({','.join(map(str, device_ids_by_type[4]))}) 
    THEN 1 ELSE 0 END) AS mechanical_ventilation
FROM `sccm-discovery.rediscover_datathon_2025.cohort_team_2` c
LEFT JOIN `sccm-discovery.rediscover_datathon_2025.device_exposure` de
  ON c.person_id = de.person_id
  AND c.visit_occurrence_id = de.visit_occurrence_id
  AND de.device_concept_id IN ({','.join(map(str, device_concept_ids))})
  AND de.device_concept_id NOT IN ({','.join(map(str, EXCLUDED_DEVICE_CONCEPT_IDS))})
GROUP BY c.person_id, c.visit_occurrence_id, c.race_name, c.gender_name
ORDER BY c.person_id, c.visit_occurrence_id
"""

print("Running fixed query...")
devices_fixed = run_query(query_fixed)
print(f"Fixed query result size: {len(devices_fixed)}")
print(f"Original cohort size: {len(cohort)}")
print(f"Should match: {len(devices_fixed) == len(cohort)}")

Running fixed query...
Fixed query result size: 99056
Original cohort size: 99056
Should match: True


## Upload

In [116]:
query = f"""
SELECT person_id, visit_occurrence_id, nasal_canula_mask, hiflo_oximyzer, cpap_bipap, mechanical_ventilation
FROM devices_fixed
"""
devices_cohort_final = duckdb.sql(query).df()

# Upload the dataframe to BigQuery
job = client.load_table_from_dataframe(devices_cohort_final, "sepsis-nlp.team_2.cohort_devices")

# Wait for the job to complete
job.result()

LoadJob<project=sepsis-nlp, location=US, id=30833ba5-cde4-4af7-afa2-734bc6e744dc>

## By race

In [113]:
query = f"""
SELECT 
    race_name, gender_name,
    AVG(nasal_canula_mask) as avg_nasal_canula_mask,
    AVG(hiflo_oximyzer) as avg_hiflo_oximyzer,
    AVG(cpap_bipap) as avg_cpap_bipap,
    AVG(mechanical_ventilation) as avg_mechanical_ventilation
FROM devices_fixed
GROUP BY race_name, gender_name
ORDER BY race_name, gender_name
"""
devices_by_race = duckdb.sql(query).df()
devices_by_race

,race_name,gender_name,avg_nasal_canula_mask,avg_hiflo_oximyzer,avg_cpap_bipap,avg_mechanical_ventilation
0,American Indian or Alaska Native,FEMALE,0.535714,0.136905,0.023810,0.190476
1,American Indian or Alaska Native,MALE,0.558140,0.142857,0.016611,0.215947
2,Asian,FEMALE,0.481221,0.113459,0.028951,0.135368
3,Asian,MALE,0.559254,0.144474,0.039281,0.177097
4,Black,FEMALE,0.574307,0.117522,0.022434,0.171048
5,Black,MALE,0.606760,0.130720,0.022091,0.197409
6,Mixed,FEMALE,0.461538,0.062937,0.000000,0.104895
7,Mixed,MALE,0.465116,0.139535,0.000000,0.100775
8,Native Hawaiian or Other Pacific Islander,FEMALE,0.442623,0.081967,0.049180,0.155738
9,Native Hawaiian or Other Pacific Islander,MALE,0.522293,0.114650,0.012739,0.146497


# Concept ID look up

In [26]:
query = """
SELECT *
FROM `sccm-discovery.rediscover_datathon_2025.concept` 
WHERE concept_id = 44818707
"""
run_query(query)

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,44818707,EHR Detail,Type Concept,Device Type,Device Type,None,OMOP4822235,1970-01-01,2099-12-31,None


# Join

In [151]:
query = f"""
SELECT c.*, 
    de.nasal_canula_mask, de.hiflo_oximyzer, de.cpap_bipap, de.mechanical_ventilation,
    -- dr.steroid_flag, dr.narcotic_flag, dr.sedative_flag, dr.vasopressor_flag,
    pr.ecmo, pr.dialysis
FROM `sepsis-nlp.team_2.cohort_with_drugs` c
LEFT JOIN `sepsis-nlp.team_2.cohort_devices` de
    ON c.person_id = de.person_id
    AND c.visit_occurrence_id = de.visit_occurrence_id
LEFT JOIN `sepsis-nlp.team_2.cohort_procedures` pr
    ON c.person_id = pr.person_id
    AND c.visit_occurrence_id = pr.visit_occurrence_id
"""
cohort_flags = run_query(query)
cohort_flags

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,...,vasopressor_flag,ethnicity_name,site_id,site_location,nasal_canula_mask,hiflo_oximyzer,cpap_bipap,mechanical_ventilation,ecmo,dialysis
0,70926,8507,1925,1,1,1925-01-01,8516,38003564,NaN,NaN,...,1,Not Hispanic or Latino,SITE-5,NE,0,0,0,0,0,0
1,77592,8507,1925,1,1,1925-01-01,8527,38003564,NaN,NaN,...,0,Not Hispanic or Latino,SITE-6,NE,1,0,0,0,0,0
2,71018,8507,1925,1,1,1925-01-01,0,38003564,NaN,NaN,...,0,Not Hispanic or Latino,SITE-5,NE,1,0,0,0,0,0
3,57129,8507,1925,1,1,1925-01-01,8527,38003564,NaN,NaN,...,1,Not Hispanic or Latino,SITE-4,MW,0,0,0,0,0,0
4,55650,8507,1925,1,1,1925-01-01,8527,38003564,NaN,NaN,...,1,Not Hispanic or Latino,SITE-4,MW,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99051,54041,8532,2006,9,1,2006-09-01,8527,38003564,NaN,NaN,...,1,Not Hispanic or Latino,SITE-4,MW,0,0,0,0,0,0
99052,60665,8532,2006,10,1,2006-10-01,0,38003563,NaN,NaN,...,1,Hispanic or Latino,SITE-5,NE,0,0,0,0,0,0
99053,53325,8532,2006,11,1,2006-11-01,8527,38003564,NaN,NaN,...,0,Not Hispanic or Latino,SITE-4,MW,0,0,0,0,0,0
99054,1080,8532,2006,12,1,2006-12-01,38003599,38003564,NaN,NaN,...,0,Not Hispanic or Latino,SITE-1,MW,0,0,0,0,0,0


## Upload

In [152]:
# Upload the dataframe to BigQuery
job = client.load_table_from_dataframe(cohort_flags, "sepsis-nlp.team_2.cohort_flags_final")

# Wait for the job to complete
job.result()

LoadJob<project=sepsis-nlp, location=US, id=5cb3aefc-ec0d-42fd-9b81-3bd0c6d6fa7d>

In [153]:
cohort_flags.to_csv("data/cohort_flags.csv", index=False)